# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [7]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [13]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [97]:
renda_reg = smf.ols('np.log(renda) ~ C(sexo, Treatment(reference="F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia',
                     data = df).fit()
df['renda_reg'] = renda_reg.resid
renda_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sat, 10 May 2025   Prob (F-statistic):               0.00
Time:                        18:15:59   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  6.5847      0.235     28.006      0.000       6.124       7.046
C(sexo, Treatment(reference="F"))[T.M]     0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                   0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                    0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda)[T.Bolsista]                  0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda)[T.Empresário]                0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]              -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda)[T.Servidor público]          0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao)[T.Pós graduação]               0.1071      0.159      0.673      0.501      -0.205       0.419
C(educacao)[T.Secundário]                 -0.0141      0.072     -0.196      0.844      -0.155       0.127
C(educacao)[T.Superior completo]           0.0938      0.072      1.298      0.194      -0.048       0.235
C(educacao)[T.Superior incompleto]        -0.0432      0.078     -0.555      0.579      -0.196       0.109
C(estado_civil)[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil)[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil)[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil)[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia)[T.Casa]                -0.0442      0.054     -0.815      0.415      -0.150       0.062
C(tipo_residencia)[T.Com os pais]         -0.0258      0.060     -0.427      0.670      -0.144       0.093
C(tipo_residencia)[T.Comunitário]         -0.1196      0.105     -1.135      0.256      -0.326       0.087
C(tipo_residencia)[T.Estúdio]              0.0665      0.099      0.670      0.503      -0.128       0.261
C(tipo_residencia)[T.Governamental]       -0.0571      0.066     -0.865      0.387      -0.186       0.072
qtd_filhos                                -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                      0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego                              0.0617      0.001     59.398      0.000       0.060       0.064
qt_pessoas_residencia                      0.2926      0.108      2.716      0.007       0.081       0.504
==============================================================================
Omnibus:

In [77]:
renda_reg

In [103]:
# Variavel mais frequente é do sexo F

var_freq = df.mode().iloc[0]
var_freq

Unnamed: 0                         0
data_ref                  2015-01-01
id_cliente                    5573.0
sexo                               F
posse_de_veiculo               False
posse_de_imovel                 True
qtd_filhos                       0.0
tipo_renda               Assalariado
educacao                  Secundário
estado_civil                  Casado
tipo_residencia                 Casa
idade                           40.0
tempo_emprego               4.216438
qt_pessoas_residencia            2.0
renda                         728.96
renda_reg                  -0.279173
Name: 0, dtype: object

In [109]:
renda_reg2 = smf.ols('np.log(renda) ~ C(sexo, Treatment(reference="F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia',
                     data = df).fit()
df['renda_reg2'] = renda_reg2.resid
renda_reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     600.2
Date:                Sat, 10 May 2025   Prob (F-statistic):               0.00
Time:                        18:27:30   Log-Likelihood:                -13666.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12415   BIC:                         2.745e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  6.7005      0.220     30.511      0.000       6.270       7.131
C(sexo, Treatment(reference="F"))[T.M]     0.7715      0.015     52.631      0.000       0.743       0.800
posse_de_veiculo[T.True]                   0.0565      0.014      3.985      0.000       0.029       0.084
posse_de_imovel[T.True]                    0.0868      0.014      6.288      0.000       0.060       0.114
C(estado_civil)[T.Separado]                0.3096      0.112      2.763      0.006       0.090       0.529
C(estado_civil)[T.Solteiro]                0.2533      0.110      2.309      0.021       0.038       0.468
C(estado_civil)[T.União]                  -0.0281      0.025     -1.110      0.267      -0.078       0.022
C(estado_civil)[T.Viúvo]                   0.3317      0.116      2.850      0.004       0.104       0.560
qtd_filhos                                -0.2443      0.109     -2.247      0.025      -0.457      -0.031
idade                                      0.0046      0.001      5.783      0.000       0.003       0.006
tempo_emprego                              0.0611      0.001     59.136      0.000       0.059       0.063
qt_pessoas_residencia                      0.2677      0.109      2.466      0.014       0.055       0.480
==============================================================================
Omnibus:                        1.331   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.514   Jarque-Bera (JB):                1.311
Skew:                           0.024   Prob(JB):                        0.519
Kurtosis:                       3.013   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Models

Analyzing the two models, we can conclude that there is no significant change after the changes.
R-squared, Adjust R-squared and AIC remain practically the same.